<a href="https://colab.research.google.com/github/wvs2/django-easy-pdf/blob/master/experimentos/Oracle/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
! pip install zeugma

In [0]:
import pandas as pd
from nltk.stem.porter import PorterStemmer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from unicodedata import normalize
from nltk import word_tokenize
import nltk

In [0]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from vector_functions import TfidfEmbeddingVectorizer, get_word2vec, MySentences, MeanEmbeddingVectorizer, MyTokenizer

In [75]:
from zeugma import EmbeddingTransformer, TextsToSequences, Padder, ItemSelector

Using TensorFlow backend.


In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [76]:
embedding = EmbeddingTransformer('glove-twitter-50')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
df_train = pd.read_csv('tweets_union_train.csv')
df_test = pd.read_csv('tweets_union_test.csv')
df_val = pd.read_csv('tweets_union_val.csv')

In [0]:
stemmer = PorterStemmer()

In [0]:
def pre_processing(tweet_text):
    # Removendo URLS
    new_text = re.sub(r"http\S+", "", tweet_text)
    # Removendo RT
    new_text = re.sub('RT @[\w_]+: ', '', new_text)
    # Removendo tags
    new_text = re.sub(r"@\S+", "", new_text)
    # Retirando caracteres especiais
    new_text = normalize('NFKD', new_text).encode('ASCII', 'ignore').decode('ASCII')
    new_text = re.sub('\s+', ' ', new_text)
    return new_text


def tokenize_with_stemmer(sentence):
    tokens = [stemmer.stem(w) for w in word_tokenize(sentence)]
    return tokens

In [0]:
for i, item in df_val.iterrows():    
    df_train = df_train.append({'tweet': item['tweet'], 'class':item['class']}, ignore_index=True)

In [0]:
train, class_train = df_train['tweet'].fillna(' ').apply(pre_processing), df_train['class']

In [0]:
test, class_test = df_test['tweet'].fillna(' ').apply(pre_processing), df_test['class']

In [66]:
w2v = get_word2vec(
    MySentences(
        train,
    ),
    'w2v_model_baseline',
    1
)

w2v_model_baseline not found. training model
Model done training. Saving to disk


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v)

In [0]:
vect_cv = CountVectorizer(analyzer='word', lowercase=True, tokenizer=tokenize_with_stemmer, stop_words='english')


In [0]:
vect_tfidf = TfidfVectorizer(
    analyzer='word', lowercase=True, tokenizer=tokenize_with_stemmer, use_idf=True, stop_words='english'
)

In [0]:
pipe_cv = Pipeline([
    ('vect', vect_cv),
    ('clf', SVC(kernel='linear', random_state=0))
])

In [0]:
pipe_tfidf = Pipeline([
    ('vect', vect_tfidf),
    ('clf', SVC(kernel='linear', random_state=0))
])

In [0]:
pipe_w2v = Pipeline([
    ('vect', mean_embedding_vectorizer),
    ('clf', SVC(kernel='linear', random_state=0))
])

In [0]:
svm = SVC(kernel='linear', random_state=0)
pipe_glove = make_pipeline(embedding, svm)

In [28]:
pipe_cv.fit(train, class_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_with_stemmer at 0x7f6719359950>,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
               

In [29]:
pipe_tfidf.fit(train, class_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_with_stemmer at 0x7f6719359950>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     de

In [69]:
pipe_w2v.fit(train, class_train)

Pipeline(memory=None,
         steps=[('vect',
                 <vector_functions.MeanEmbeddingVectorizer object at 0x7f67187667b8>),
                ('clf',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=0, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [79]:
pipe_glove.fit(train, class_train)

Pipeline(memory=None,
         steps=[('embeddingtransformer',
                 EmbeddingTransformer(aggregation='average',
                                      model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f6716cb62e8>)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=0, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [0]:
y_pred_cv = pipe_cv.predict(test)

In [0]:
y_pred_tfidf = pipe_tfidf.predict(test)

In [0]:
y_pred_w2v = pipe_w2v.predict(test)

In [0]:
y_pred_glove = pipe_glove.predict(test)

In [0]:
def cont_acertos(*args):
  a, b, c, d = args
  cont = 0
  for k, v in class_test.items():
    if a[k] == v or b[k] == v or c[k] == v or d[k] == v:
      cont += 1
  return cont

In [82]:
print(cont_acertos(y_pred_cv, y_pred_tfidf, y_pred_w2v, y_pred_glove))

4774
